In [1]:
import torch
from torch.utils.data import DataLoader

from model import *
from trainer import *
from database import FlowData_B as FlowData
from config import ENV_CON_B as ENV_CON

In [2]:
forecast_step = 1
lr, max_step = 0.001, 100
hidden_dim = 16
block_index = 2
re_index = 4

seed = torch.initial_seed()
net = MNSTFN(
    in_dim=2,
    hidden_dim=hidden_dim,
    forecast_step=forecast_step,
    block_index= block_index,
    re_index= re_index
)
recoder = ExpRecorder(
    save_path=ENV_CON['path_e'],
    model_name=f"{net.__class__.__name__}-{ENV_CON['data_name']}",
    model_file_name= f'{net.__class__.__name__}-{forecast_step}'
    )

In [ ]:
recoder.record('\n--------------start-----------------\n')
recoder.record(f'model name:{net.__class__.__name__}\n\n')
recoder.record(f'lr:{lr}    max_tsep:{max_step}    forecast:{forecast_step}    current random seed:{seed}')
recoder.record(f'hidden dim:{hidden_dim}    block index:{block_index}   re index:{re_index}')

In [ ]:
data_set = FlowData(forecast_step)
test_size = 48*7*4
train_size = len(data_set) - test_size
train_dataset = torch.utils.data.Subset(data_set,[i for i in range(0,train_size)])
test_dataset = torch.utils.data.Subset(data_set,[i for i in range(train_size,len(data_set))])
train_loader = DataLoader(train_dataset,32,shuffle=True)
# train_loader = DataLoader(train_dataset,32,shuffle=True,drop_last=True)
test_loader = DataLoader(test_dataset,1,shuffle=False)
recoder.record(f'dateset length:{len(data_set)}')
recoder.record(f'train data length:{train_size} test data length:{test_size}')

In [ ]:
if torch.cuda.is_available():
    net = net.cuda()

train_net(net,train_loader,test_loader,max_step,lr,ENV_CON['scale'],recoder)